# Characters Degrees of Separation 

## Setup

In [3]:
import pandas as pd, matplotlib.pyplot as plt, numpy as np
import os, inspect, graphviz, pickle

In [4]:
os.listdir('../')

['.ipynb_checkpoints',
 'setup_virtual_environment_for_streamlit_app.sh',
 'log.txt',
 '.gitignore',
 '__pycache__',
 'notebooks',
 'app.py',
 'README.md',
 'debug_streamlit.ipynb',
 '.git',
 'requirements.txt',
 'venv',
 'assets',
 'rongzi.py',
 'data']

In [5]:
ccd = pd.read_csv('../data/WikiCCD_2021-07-18.csv')
ccd = ccd.loc[ccd.index.difference([3618, 21004, 21100])]  # remove some duplicates
ccd = ccd.set_index('Component')

#### make a dict for strings found as "parents" that have a record as components (and the ones that don't)

In [23]:
ccd.columns

Index(['Strokes', 'CompositionType', 'LeftComponent', 'LeftStrokes',
       'RightComponent', 'RightStrokes', 'Signature', 'Notes', 'Section'],
      dtype='object')

In [70]:
all_parents = set(ccd.LeftComponent.values).union(set(ccd.RightComponent.values))
all_components = set(ccd.index.values)
no_record_set = all_parents.difference(all_components)
no_record_ser = ccd.RightComponent.map(lambda x: x in no_record_set).astype(int) + ccd.LeftComponent.map(lambda x: x in no_record_set).astype(int)
no_record_df = ccd.copy().loc[no_record_ser > 0]
no_record_df['no_record'] = no_record_ser

In [77]:
print(no_record_set)

{nan, '臼同', '⺌', '尤旨', '㇇', '㩅', '王缶', '山分', '𠨐', '䍛', '丷十', '𫠣', '日虫虫', '𤾡', '田疋', '丁丷', '宀心', '冘皿', '爫无', '执云', '田凵', '芮*', '土示', '口了又', '𨥫', '干干', '彐+人', '夕寸', '豆寸', '井?', '𡴎', '目大良', '袁*', '聂?', '正見', '山並', '艹土', '卄両', '㐬皿', '罒?寸', '𥘈', '疌 ', '乛耳', '口川', '广艹', '禾尤旨', '女 ', '⺊冖釆', '一亻', '儿*', '廿北口', '⺈内', '⺳', '工几', '人镸', '廌去', '穴土', '备录', '关黑', '𥏫*', '自亼冊', '穴方', '冫冫', '从止', '丿?夂', '貝貝', '*', '厶大', '士冖禾', '八田', '苜*', '刀牛', '水水', '龶', '皀殳', '丆且分', '免冫', '北白', '仌十仌', '申乁', '艹巾', '小一', '匚一', '𥃲', '艹品', '𣄼', '爿喬', '爫击?', '人*', '罒方', '目乚', '舟巜', '䩗', '口冊', '日夨', '臼?由', '戊貝', '口口田一', '2旧', '口勹 又', '彳枼', '巛歹刂 ', '𠠵', '川凶', '占又', '𢌳', '菐*', '林月', '爿丙', '免生', '日匂', '𠁳', '䖒', '田一', '彳支', '广?', '戊贝', '<夓', '𥝢', '厂二', '八十', '辰?', '彐彐彐', '幺白幺', '㘸', '𣳾', '幺幺大', '𧘇', '右*比', '敄目', '一由', ' 丯口', '丑刂', '鳥*', '丷干', '𧆨', '一巛', '幺月', '兔生', '戌小', '目兆', '氵刅', '夂?夂', '月又', '月寸', '口十', '丬夕', '爿臬', '禾魚', '⺈', '㇏', '𡉀', '𠱠', '丷人', '负*', '厃儿', '丶囗釆', '一亚', '爫糸', '7儿', '穴瓜瓜瓜', '乚米', '丶凡', '舌自', '尸示', '⺆', '𤋱',

In [ ]:
len(no_record_set)

444

In [ ]:
len(all_components)

21166

In [78]:
ccd.loc[ccd.RightComponent == '自亼冊']

,Strokes,CompositionType,LeftComponent,LeftStrokes,RightComponent,RightStrokes,Signature,Notes,Section
Component,,,,,,,,,
癟,19,吕,疒,5,自亼冊,14,KHUB,/?,疒


#### As a result of this EDA, I'm going to do three things:
1. Drop the asterisk (*) and question mark (?) automatically.
2. Split records by character as that will yield a few extra records, and it looks like these multiple-component entries are kinda interesting.
3. Continue to ignore parent components that don't have their own record.

## get and graph vertical family trees

### get all ancestors

In [6]:
def get_all_ancestors(comp:str) -> list[str]:
    """
        INPUTS: character component as str
        OUTPUTS: list of all ancestor components
    """
    # all-ancestors stack of characters
    aa_stack = [comp]
    aa_list = [comp]
    
    while aa_stack:
        c = aa_stack.pop(0)
        
        parents = get_parents_lr(c)
        for l_r in parents:
            if parents[l_r]:
                aa_stack.append(parents[l_r])
                aa_list.append(parents[l_r])
    return aa_list

#### child functions

In [7]:
def get_parents_lr(comp:str) -> dict:
    """
        if:
            - the decomposed-/parent-component is different than its composite-child, and
            - the decomposed-/parent-component has its own record,
        then:
            - show the decomposed-/parent-component as the dict value,
        else:
            - show None
    """
    [lc, rc] = ccd.loc[comp][['LeftComponent','RightComponent']].to_list()
    
    if lc != comp:
        l = check_for_parents(lc)
    else:
        l = None
        
    if rc != comp:
        r = check_for_parents(rc)
    else:
        r = None
    
    return {'LeftComponent': l, 'RightComponent': r}

In [8]:
def get_parents(c:str) -> list:
    return pdb[c]

In [9]:
def check_for_parents(comp:str) -> int:
    """
        If a string is a component with its own record of parents, 
            then return that index. 
        Else,
            return None.
        (This function is used to build get_parents_lr)
    """
    try:
        return ccd.loc[comp].name
    except KeyError:
        return None

In [10]:
def make_parents_db() -> dict:
    """
    Create database of immediate parent-components, for quick searching in that direction.
    """
    allcomps = ccd.index
    parents_db = {c: [] for c in allcomps}
    for c in ccd.index:
        parents_db[c] = list(get_parents_lr(c).values())
    return parents_db

### get kids 

In [11]:
def get_all_descendents(comp:str) -> dict:
    """
        INPUTS: target component
        OUTPUTS: list of all descendant components
    """
    ad_stack = [comp]
    ad_list = [comp]

    while ad_stack:
        c = ad_stack.pop(0)
        kids = kdb[c]
        for kid in kids:
            ad_stack.append(kid)
            ad_list.append(kid)
    
    return ad_list

#### child functions

In [12]:
def get_kids():
    return kdb[c]

In [13]:
def make_kids_db() -> dict:
    """
    Create database of immediate children-components, for quick searching in that direction.
    """
    allcomps = ccd.index
    kids_db = {c: [] for c in allcomps}
    for c in ccd.index:
        comps = ccd.loc[c][['LeftComponent','RightComponent']].to_list()
        for xc in comps:
            if xc == c:
                continue
            try:
                kids_db[xc].append(c)
            except:
                pass
    return kids_db

### graph

In [85]:
def vertical_family_tree(comp:str, directory:str):
    dot = graphviz.Digraph(comment='vertical family tree')
    aa = get_all_ancestors(comp)
    for c in aa:
        add_nodes_and_edges_parents(c, dot)

    add_nodes_and_edges_kids(comp, dot)
    
    dot.engine = 'neato'
    dot.attr('graph', overlap='false')
    
    if 'Digraph.gv.pdf' in os.listdir(directory):
        os.remove(directory + 'Digraph.gv.pdf')
    
    filename = dot.render(format='png', directory=directory)
    
    return dot

In [15]:
def add_nodes_and_edges_parents(comp:str, dot:graphviz.Digraph):
    [lc, rc] = ccd.loc[comp, ['LeftComponent', 'RightComponent']].tolist()
    
    dot.node(comp)
    
    if comp == lc:
        return None
    
    parents = get_parents_lr(comp)
    if parents['LeftComponent']:
        dot.edges([f"{lc}{comp}"])
    if parents['RightComponent']:
        dot.edges([f"{rc}{comp}"])
    return None

In [16]:
def add_nodes_and_edges_kids(comp:str, dot:graphviz.Digraph):
    dot_body_joined = ''.join(dot.body)
    dot_body_charset = set(dot_body_joined)
    comp_and_kids = kdb[comp] + [comp]
    for c in comp_and_kids:
        if c not in dot_body_charset:
            dot.node(c)
    for c in kdb[comp]:
        dot.edges([f"{comp}{c}"])

### get_and_graph

###### get kids and parents databases and pickle

In [17]:
kdb = make_kids_db()
pdb = make_parents_db()

In [18]:
with open('../assets/ccd_pdb_kdb.pickle', 'wb') as f:
    pickle.dump((ccd, pdb, kdb), f)

###### graph

In [86]:
if not os.path.isdir('../data/processed/'):
    os.makedirs('../data/processed/')
dot = vertical_family_tree('青', '../data/processed/')

In [84]:
!evince ../data/processed/Digraph.gv.pdf

In [22]:
# !display ../data/processed/Digraph.gv.svg

display-im6.q16: unable to open image `../data/processed/Digraph.gv.pdf': No such file or directory @ error/blob.c/OpenBlob/2874.


#### export pdf

## connect two trees (tt)

##### designing scorefunc

In [ ]:
def score(comp:str) -> int:
    strokes = ccd.loc[comp].Strokes
    epsilon = 0.1 / ord(comp)
    return scorefunc(strokes) + epsilon

def scorefunc(strokes:int):
    x = strokes - 6
    y1 = 0 if x < 0 else .001*x**2
    y2 = 0 if x > 0 else .07*np.exp(-x)
    return 1 + y1 + y2

In [ ]:
fig = plt.figure(figsize=(10,4))
ax = fig.gca()
stroke_count_occurence = ccd.Strokes.value_counts().sort_index()
stroke_count_occurence.map(np.log).rename('stroke-count: log-occurence').plot(ax=ax)
stroke_count = stroke_count_occurence.index.to_series()
stroke_count.map(scorefunc).rename('stroke_scores').plot(ax=ax)
max_strokes = ccd.Strokes.max()
ax.set_xlim(right=max_strokes)
# ax.set_yscale('log')
ax.legend()
plt.show()

#### let the buds sprout

###### setup

In [ ]:
kdb = make_kids_db()

In [ ]:
def look_at_next(T1):
    T1_keys = list(T1.keys())  # complete list of keys from previous state
    t1 = {} # building the new state
    for a in T1_keys:
        t1[a] = {} # initialize a dict for each of previous state's keys
        for b in (i for i in get_kids(a) + get_parents(a) if i): 
            t1[a][b] = (T1[a][0] + [b],
                        T1[a][1] + score(b))
    return t1

In [ ]:
def take_next_step(T1, t1):
    for c in t1:
        for d in t1[c]:
            try:
                T1[d]
            except KeyError as e:
                T1[d] = t1[c][d]
            if T1[d][1] > t1[c][d][1]:
                T1[d] = t1[c][d]
    return T1

###### start

In [ ]:
# starts = ('祥','愛') # [祥, 羊, 鮮, 魚, 鱫, 愛]
# starts = ('潘','祥')  # [潘, 番, 翻, 羽, 翔, 羊, 祥]
# starts = ('潘', '愛') # [潘, 番, 鐇, 金, 鑀, 愛]
# starts = ('芒', '愛') # [芒, 恾, 忄, 懓, 愛]
# starts = ('芒', '祥') # [芒, 硭, 石, 祏, 礻, 祥]

starts = ('青', '鳥')
s = starts
T1 = {s[0]: ([s[0]], 0)}
T2 = {s[1]: ([s[1]], 0)}

In [ ]:
t1 = look_at_next(T1)
t2 = look_at_next(T2)
T1 = take_next_step(T1, t1)
T2 = take_next_step(T2, t2)
print((len(T1), len(T2)))
intersection = set(T1).intersection(set(T2))
print(len(intersection))

In [ ]:
scoring = {c: (T1[c][1] + T2[c][1]) for c in intersection}
scoring = pd.Series(scoring, name='scoring').to_frame()
path = {c: (T1[c][0] + T2[c][0][:-1][::-1]) for c in intersection}
path = pd.Series(path, name='path').to_frame()
path_scores = path.join(scoring).sort_values('scoring', ascending=True)

In [ ]:
path_scores.head(10)